In [1]:
import pandas as pd
import nltk

In [2]:
df_result = pd.read_csv("data/results.csv", index_col=0)
df_result.head(1)

,movie_name,index,utterance,count_utterances,tag,we_label_fox_news,we_label_twitter,bert_label_twitter,bert_label_fox_news,lstm_label_twitter,lstm_label_fox_news,hate_keyword_count,hate_keyword_count_unique,hate_keyword,hate_keyword_sources
0,12 Angry Men,0,"Thank you very much, sir.",2065,racism,not hate speech,neither,neither,not hate speech,hate speech,not hate speech,0,0,[],[]


In [3]:
def replace_val(df):
    df = df.replace('neither',0)
    df = df.replace('not hate speech',0)
    df = df.replace('offensive language',1)
    df = df.replace('hate speech',2)
    return df

In [4]:
def calculate_keyword_formular(count):
    if count == 1:
        return 1
    elif count == 2:
        return 2
    elif count==0:
        return 0
    else:
        return 3

In [5]:
df_result['we_label_fox_news'] = replace_val(df_result['we_label_fox_news'])
df_result['we_label_twitter'] = replace_val(df_result['we_label_twitter'])
df_result['bert_label_twitter'] = replace_val(df_result['bert_label_twitter'])
df_result['bert_label_fox_news'] = replace_val(df_result['bert_label_fox_news'])
df_result['lstm_label_twitter'] = replace_val(df_result['lstm_label_twitter'])
df_result['lstm_label_fox_news'] = replace_val(df_result['lstm_label_fox_news'])

In [6]:
df_result['keyword_count_formular'] = df_result['hate_keyword_count_unique'].apply(calculate_keyword_formular)

In [7]:
df_result.head(1)

,movie_name,index,utterance,count_utterances,tag,we_label_fox_news,we_label_twitter,bert_label_twitter,bert_label_fox_news,lstm_label_twitter,lstm_label_fox_news,hate_keyword_count,hate_keyword_count_unique,hate_keyword,hate_keyword_sources,keyword_count_formular
0,12 Angry Men,0,"Thank you very much, sir.",2065,racism,0,0,0,0,2,0,0,0,[],[],0


In [8]:
df_result['combined_results'] = (df_result['we_label_fox_news']
                                    +df_result['we_label_twitter']
                                    +df_result['bert_label_twitter']
                                    +df_result['bert_label_fox_news']
                                    +df_result['lstm_label_twitter']
                                    +df_result['lstm_label_fox_news']
                                    +df_result['keyword_count_formular'])

In [9]:
df_result['combined_normlized'] = round(df_result['combined_results']/df_result['combined_results'].max(),2)

## add bins

In [10]:
df_result['combined_normlized_bin'] = pd.cut(df_result['combined_normlized'], bins=[0, .5, .8, 1], labels=['neither', 'offensive', 'hate'])

In [11]:
df_result.head(1)

,movie_name,index,utterance,count_utterances,tag,we_label_fox_news,we_label_twitter,bert_label_twitter,bert_label_fox_news,lstm_label_twitter,lstm_label_fox_news,hate_keyword_count,hate_keyword_count_unique,hate_keyword,hate_keyword_sources,keyword_count_formular,combined_results,combined_normlized,combined_normlized_bin
0,12 Angry Men,0,"Thank you very much, sir.",2065,racism,0,0,0,0,2,0,0,0,[],[],0,2,0.15,neither


# Analysis

# Creating Movie Metadata Dataframe

In [12]:
mean_hate_score_by_movie_df = df_result.groupby('movie_name').combined_normlized.mean().to_frame('mean_hate_score').reset_index().sort_values(by='mean_hate_score', ascending=False)

In [13]:
mean_hate_score_by_movie_df.head(1)

,movie_name,mean_hate_score
207,The Red Pill,0.286429


In [14]:
new_label_distribution_df = df_result.groupby(['movie_name', 'tag']).combined_normlized_bin.value_counts().to_frame('count_bin').reset_index()
new_label_distribution_df.head(3)

,movie_name,tag,combined_normlized_bin,count_bin
0,12 Angry Men,racism,neither,1959
1,12 Angry Men,racism,offensive,30
2,1917,friendship,neither,1408


In [15]:
new_label_distribution_df[new_label_distribution_df.movie_name=='friendship']

,movie_name,tag,combined_normlized_bin,count_bin


In [16]:
df = new_label_distribution_df.groupby(['tag', 'combined_normlized_bin']).count_bin.sum().to_frame('count').reset_index()
df_pivot = pd.pivot_table(
    df,
    values="count",
    index="tag",
    columns='combined_normlized_bin'
)

In [17]:
ax = df_pivot.plot(kind='bar', title=title)

NameError: name 'title' is not defined

In [ ]:
new_label_distribution_df.groupby(['tag', 'combined_normlized_bin']).count_bin.sum().to_frame('sum').reset_index()

In [ ]:
a = df_result.groupby(['tag']).combined_normlized_bin.value_counts(normalize=True).to_frame('percentage').reset_index()

In [ ]:
a

In [ ]:
a[a.tag=='friendship'].percentage.plot(kind='pie', autopct='%1.0f%%')

In [ ]:
a[a.tag=='hate_speech'].percentage.plot(kind='pie', autopct='%1.0f%%')

In [ ]:
a[a.tag=='racism'].percentage.plot(kind='pie', autopct='%1.0f%%')

In [ ]:
ax = df_result.groupby(['tag']).combined_normlized_bin.value_counts(normalize=True).unstack().plot(kind='bar', stacked = True, title=title, color=['orange', 'green', 'blue'])
for i, p in enumerate(ax.patches):
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy() 
    a = round(x + width/2, 2)
    b = round(y + height*1.00, 2)
    height = round(height, 2)
    ax.annotate(f'{height}', (a, b), ha='center')
ax.legend(bbox_to_anchor=(1.5, 1))

In [ ]:
new_label_distribution_df[new_label_distribution_df.combined_normlized_bin=='hate'].groupby('tag').count_bin.mean()

In [ ]:
new_label_distribution_df[new_label_distribution_df.combined_normlized_bin=='offensive'].groupby('tag').count_bin.mean()

In [ ]:
new_label_distribution_df[new_label_distribution_df.combined_normlized_bin=='neither'].groupby('tag').count_bin.mean()

In [ ]:
new_label_distribution_df[new_label_distribution_df.combined_normlized_bin=='hate'].sort_values(by='count_bin', ascending=False).head(10)

In [ ]:
new_label_distribution_df[new_label_distribution_df.combined_normlized_bin=='offensive'].sort_values(by='count_bin', ascending=True).head(10)

In [ ]:
movie_metadata = df_result.drop_duplicates(subset=['movie_name'])[['movie_name', 'count_utterances', 'tag']]

In [ ]:
movie_metadata = movie_metadata.merge(mean_hate_score_by_movie_df, on='movie_name')

In [ ]:
movie_metadata.sort_values(by='mean_hate_score', ascending=False).head(20)

# Mean hate_score of movies distributed by tags

In [ ]:
movie_metadata.groupby('tag').mean_hate_score.max()

In [ ]:
movie_metadata.groupby('tag').count_utterances.mean()

In [ ]:
movie_metadata.tag.value_counts()

In [ ]:
ax = movie_metadata.groupby('movie_name').mean_hate_score.max().sort_values(ascending=False).plot(kind='bar', figsize=(30,5))
ax.set_ylim([0, 1])

In [ ]:
ax = movie_metadata.groupby('movie_name').mean_hate_score.max().sort_values(ascending=False).plot(kind='line', figsize=(30,5))
ax.set_ylim([0, 1])